## 4.1 RNN(recurrent neural network)

### **特點一：有記憶的神經網路。**
  有別於以前的神經網路，不分順序相同筆資料輸入會得到一樣的值。
  但RNN會記憶先前輸出結果，並回傳當成新的輸入。

* 例子：對話機器人

## 4.2 RNN的運用
* 前次輸出與當次輸入各自權重，並加上偏值
![RNN](4_1.jpg)

* **限制：標準的RNN可能會遇到每次調整，造成難以訓練。因此現在操作上的RNN可能指的是以下兩種改良方式。**
    - LSTM(Long Short Term Memory)
    - GRU(Gated Recurrent Unit)
    
### **特點二：輸入值與輸出值可以為不同的類型的資料，如輸入圖片並輸出文字。**

* 例子：
    - 翻譯
    - 生成影片描述
    - 生成一段文字
    - 完成一半的圖
    - Andrej Karpathy(RNN有名推廣者)使用RNN模仿教科書與莎翁作品
    


## 4.3 RNN的應用：情意分析

#### 情意分析：分析文本與評語，進行正負評的分類。
    1. 問題：將一段文字透過函數學習，即可判斷內容為正負評。
    2. 函數形式：文字段落-f()-正負評
    3. 訓練資料：keras內有IMDB影評資料，訓練與測試資料各2.5萬筆。
    4. 架構神經網路：
        一萬個字=>X(embedding)=>RNN(150個RNN cell)=> D(1維數字)
    5. 學習法：ADAM

## 4.4 做情意分析RNN神經網路

### (1)初始準備

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

### (2)讀入IMDB資料庫

In [3]:
from keras.datasets import imdb

* num_words可以篩選文字出現的頻率

In [4]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 9s 1us/step


In [5]:
len(x_train)

25000

In [6]:
len(x_test)

25000

### (2.1)輸入資料
這裡顯示的是文字編碼，在以下例子為1為出現頻率最高的字。

In [8]:
x_train[99]

[1,
 1230,
 3765,
 566,
 97,
 189,
 102,
 86,
 7,
 32,
 4,
 973,
 16,
 55,
 355,
 18,
 14,
 20,
 4,
 64,
 542,
 173,
 16,
 4,
 893,
 2115,
 5376,
 250,
 39,
 8013,
 4,
 1362,
 2,
 14,
 102,
 47,
 57,
 599,
 633,
 6,
 1317,
 2,
 8,
 6,
 189,
 20,
 57,
 206,
 57,
 116,
 5,
 57,
 836,
 82,
 6,
 1317,
 2,
 3728,
 2,
 9,
 6,
 52,
 284,
 21,
 29,
 9,
 38,
 2245,
 5,
 1044,
 11,
 14,
 15,
 45,
 619,
 50,
 71,
 6,
 171,
 531,
 15,
 71,
 424,
 8,
 30,
 163,
 6211,
 4,
 1629,
 189,
 212,
 102,
 5,
 57,
 31,
 1498,
 11,
 4,
 311,
 13,
 197,
 15,
 14,
 20,
 16,
 1150,
 1479,
 5,
 13,
 161,
 990,
 692,
 5,
 1706,
 12,
 69,
 77,
 1194,
 8,
 3245,
 2001,
 553,
 67,
 14,
 20,
 48,
 25,
 423,
 13,
 131,
 124,
 51,
 25,
 122,
 236,
 1506,
 198,
 4,
 64,
 552,
 7,
 415,
 37,
 62,
 169,
 14,
 20,
 60,
 2602,
 629,
 5,
 615,
 14,
 9,
 8,
 25,
 1230,
 3765,
 570,
 231,
 189,
 102,
 14,
 20,
 166,
 2039,
 168,
 40,
 2450,
 5486,
 3298]

* 可以發現每筆影評長度並不一致，有待調整。

In [9]:
for i in range(10):
    print(len(x_train[i]), end=',')

218,189,141,550,147,43,123,562,233,130,

### (2.2) 輸出資料部分

In [10]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

### (2.3) 輸入前的資料處理
* RNN可以接受不同長度的輸入，但仍要
    - 文字長度要設立上限
    - 把每段文字弄成一樣長，太短的補上0
* keras會再將此輸入值轉成10000維向量

In [11]:
from keras.preprocessing import sequence

In [12]:
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

### (3.1)建構神經網路

    * embedding：將10000維向量壓到128維
    * 用128個LSTM
    * 最後output使用sigmoid送出。

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [14]:
model= Sequential()

In [15]:
model.add(Embedding(10000,128))

In [16]:
model.add(LSTM(150))

In [17]:
model.add(Dense(1, activation='sigmoid'))

### (3.2) 組裝

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               167400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 151       
Total params: 1,447,551
Trainable params: 1,447,551
Non-trainable params: 0
_________________________________________________________________


In [20]:
# LSTM 167400 3個gate*(128維+150個神經元參數+偏值)+...
3*(128+150+1)*150 + (128+150+1)*150

167400

In [24]:
model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

## (4)訓練

In [26]:
model.fit(x_train, y_train,
         batch_size=32,
         epochs=15)

C:\Users\Angus\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
25000/25000 [==============================] - 254s 10ms/step - loss: 0.4347 - accuracy: 0.7938
Epoch 2/15
25000/25000 [==============================] - 234s 9ms/step - loss: 0.2677 - accuracy: 0.8919
Epoch 3/15
25000/25000 [==============================] - 230s 9ms/step - loss: 0.1854 - accuracy: 0.9292
Epoch 4/15
25000/25000 [==============================] - 226s 9ms/step - loss: 0.1316 - accuracy: 0.9512
Epoch 5/15
25000/25000 [==============================] - 226s 9ms/step - loss: 0.0910 - accuracy: 0.9688
Epoch 6/15
25000/25000 [==============================] - 225s 9ms/step - loss: 0.0701 - accuracy: 0.9755
Epoch 7/15
25000/25000 [==============================] - 312s 12ms/step - loss: 0.0475 - accuracy: 0.9844
Epoch 8/15
25000/25000 [==============================] - 261s 10ms/step - loss: 0.0421 - accuracy: 0.9872
Epoch 9/15
25000/25000 [==============================] - 226s 9ms/step - loss: 0.0258 - accuracy: 0.9922
Epoch 10/15
25000/25000 [==================

## (5)檢視結果

In [27]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 57s 2ms/step


In [28]:
print("測試資料的loss:", score[0])
print("測試資料的正確率:", score[1])

測試資料的loss: 1.0617911480247975
測試資料的正確率: 0.830079972743988


In [29]:
model_json= model.to_json()
open('imdb_model_rnn.json', 'w').write(model_json)
model.save_weights('imdb_rnn_weights.h5')

### 另一種儲存神經網路的方式

In [30]:
model.save('myrnn.h5')